# RAG с текстовым поиском на основе Yandex Search API

В данном примере рассмотрим возможность построения RAG с использованием текстового поиска на основе [Yandex Search API](https://yandex.cloud/ru/services/search-api)  

Для работы с примером нужно:

- Получить [ключ авторизации](https://developers.sber.ru/docs/ru/gigachat/quickstart/ind-using-api#poluchenie-avtorizatsionnyh-dannyh) GigaChat API.
- Получить идентификатор каталога (folderid) и API-ключ (apikey) для [аутентификации в сервисе](https://yandex.cloud/ru/docs/search-api/operations/auth) Yandex Search API. 

## Установка зависимостей

Для работы примера установите зависимости:

In [ ]:
!pip install langchain-gigachat python-dotenv -q

## Авторизация в GigaChat API

Для работы к моделям GigaChat, нужно передать ключ авторизации GigaChat API.

Ключ можно указать в переменной среды `GIGACHAT_CREDENTIALS`, заданной в файле `.env` или созданной с помощью команды:

```sh
export GIGACHAT_CREDENTIALS=ключ_авториазации
```

При инициализации проверяется наличие переменной среды `GIGACHAT_CREDENTIALS` с заданным ключом авторизации GigaChat API.
Если переменная отсутствует, вы сможете указать ключ в поле **Введите ключ авторизации GigaChat API**.

О способах авторизации и поддерживаемых переменных среды — в [README библиотеки gigachat](https://github.com/ai-forever/gigachat).

In [ ]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Введите ключ авторизации GigaChat API: ")

## Авторизация в Yandex Search API

Добавьте функцию, которая поможет создать переменные для авторизации в Yandex Search API:

In [2]:
def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass(f"{key}:")

Укажите данные для авторизации в Yandex Search API:

In [ ]:
_set_env("YANDEX_API_KEY")
_set_env("YANDEX_FOLDER_ID")

## Конфигурация

In [4]:
from operator import itemgetter
from textwrap import dedent

from IPython.display import Markdown
from langchain_gigachat.chat_models import GigaChat
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel

from yandex_search import YandexSearchAPIRetriever, YandexSearchAPIWrapper

Определим дополнительные функции для форматирования документов до отправки их в LLM (`format_docs`) и для формирования финального ответа (`generate_final_answer`).

In [5]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])


def generate_final_answer(response):
    final_answer_template = dedent(
        """
        ### Вопрос
        {question}
        
        ### Ответ
        {answer}
        
        ### Источники
        {sources}"""
    ).strip()

    final_answer = final_answer_template.format(
        question=response["question"],
        answer=response["answer"],
        sources="\n\n".join(
            f'{doc.page_content} ({doc.metadata["url"]})' for doc in response["context"]
        ),
    )

    return final_answer

Используем модель `GigaChat-Pro` для данной задачи.

In [6]:
model = GigaChat(
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)

Создадим клиента для Yandex Search API с параметрами по умолчанию.  
Детальную информацию по дополнительным параметрам можно уточнить в [документации](https://yandex.cloud/ru/docs/search-api/concepts/get-request).

In [8]:
api_wrapper = YandexSearchAPIWrapper()

Настроим retriever на основе созданного клиента, получая первые k наиболее релевантных ответов на запрос.  
В случае, если клиент для Yandex Search API используется с параметрами по умолчанию, то можно не передавать аргумент api_wrapper для retriever.

In [10]:
retriever = YandexSearchAPIRetriever(api_wrapper=api_wrapper, k=30)

Создадим chain для генерации ответа от LLM с возможностью получения информации по документам, которые использовались при формировании ответа.

In [11]:
QA_TEMPLATE = """Ответь на следующий вопрос на основе предоставленного контекста:

{context}

Вопрос: {question}
Ответ: 
"""
prompt = ChatPromptTemplate.from_template(QA_TEMPLATE)

output_parser = StrOutputParser()

chain_without_source = (
    RunnableParallel(
        {
            "context": itemgetter("context") | RunnableLambda(format_docs),
            "question": itemgetter("question"),
        }
    )
    | prompt
    | model
    | output_parser
)
chain_with_source = RunnableParallel(
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
).assign(answer=chain_without_source)

## Примеры использования

In [14]:
query = "Какие концерты проходят в Москве в феврале 2025 года?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Какие концерты проходят в Москве в феврале 2025 года?

### Ответ
В феврале 2025 года в Москве запланировано множество концертов, охватывающих различные музыкальные жанры и площадки. Вот некоторые из ключевых событий:

1. **Концерт Эмина** — 8 февраля 2025 года, Театр музыки и шоу «III Рим». Это мероприятие обещает стать одним из ярких событий месяца.
   
2. **Юбилейный концерт Государственного камерного оркестра «Виртуозы Москвы»** — 9 февраля 2025 года, Большой зал Московской консерватории. Дирижером выступит известный маэстро Владимир Спиваков.

3. **Концерт группы «Звери»** — дата уточняется, но известно, что коллектив отпразднует свое 25-летие серией выступлений, одно из которых пройдет в Москве.

4. **Тур группы «Снайперы» Дианы Арбениной** — в феврале 2025 года запланированы стадионные шоу в нескольких городах, включая Москву.

5. **Концерты хип-хоп музыкантов** — такие исполнители, как Mishouris Blues Band, Авиа, Звери, Blues Cousins и другие, также планируют выступления в московских клубах.

Кроме того, в феврале продолжатся мероприятия, посвященные Китайскому Новому году, а также начнется празднование Масленицы, которое традиционно сопровождается культурными событиями.

Это лишь малая часть того, что будет происходить в музыкальной жизни Москвы в феврале 2025 года. Для получения полного расписания рекомендуется обращаться к специализированным сайтам продажи билетов и афишам мероприятий.

### Источники
от 3 до 6 лет.
В случае, если третье лицо (правообладатель или уполномоченное им лицо) считает, что его права на объект интеллектуальной собственности нарушаются, он может направить претензию по адресу: 109147, г. Москва, вн.тер.г. муниципальный округ Таганский, ул. Воронцовская, д. 1/3, стр. 2А и по e-mail: support@ticketland.ru. (https://www.ticketland.ru/concert/2025/2/)

Купить билеты на Концерты в Москве 6–28 февраля 2025 на Яндекс Афише. (https://afisha.yandex.ru/moscow/concert/feb)

Купить билеты на концерт в Москве на февраль 2025, афиша концертов и цены на них. (https://www.afisha.ru/msk/schedule_concert/na-fevral/)

На этой странице перечислены концерты в Москве. Афиша и расписание выступлений артистов в феврале 2025 года которые проходят сегодня, либо будут проходить скоро: например завтра, на этих выходных и т.д. Кудамоскоу в курсе 6 музыкальных событий и концертов, которые пройдут в Москве в феврале 2025 года. (https://kudamoscow.ru/event/concert/02-2025/)

Концерт Эмина Концерт Эмина 8 февраля 2025, 19:00 Театр музыки и шоу «III РИМ».
Самые интересные концерты в Москве. Классическая музыка, вечеринки, концерты под открытым небом и в залах - мир концертов Москвы необыкновенно разнообразен. КП-Афиша - удобный путеводитель для тех, кто собирается выбрать концерт по душе. (https://www.kp.ru/afisha/msk/konczerty/fevral/)

Концерты в России февраль 2025: масштабные шоу и юбилейные выступления. Февраль 2025 насыщен яркими музыкальными событиями. В Москве группа "Звери" отметит 25-летие акустической программой.
В этом году команда отмечает свое 25-летие серией концертов и один из них состоится в Москве уже скоро. (https://concierge-enjoy.com/news/tpost/aezz1j0x11-kontserti-v-rossii-fevral-2025-masshtabn)

В 2025 году тур рок-снайперов Дианы Арбениной продолжается — в феврале их стадионные шоу пройдут в Нижнем Новгороде и Новосибирске. Позже группу дождутся в Казани, Самаре, Оренбурге и Калининграде.
В прошлом году МЧТ прервал молчание длиной в 5 лет, выпустив альбом WAFA. Хип-хоп-мантры оттуда прозвучат в столичных клубах Base в начале и конце февраля. (https://live.mts.ru/lifestyle/article/kuda-shodit-kontserti-fevralya-2025)

Концерты в феврале 2025. 1 февр.
[20000 ₽] — К юбилею Государственного камерного оркестра «Виртуозы Москвы» Дирижер — Владимир Спиваков 9 февраля в Большом зале Московской консерватории состоится юбилейный концерт одного из самых известных коллективов России — Государственного камерного оркестра «Виртуозы Москвы» (художественный. (https://msk.zeroevent.ru/2025-02?category=concert)

Билеты на лучшие концерты в Москве в Феврале 2025 года. Успейте купить заранее по низким ценам! (https://concert.moscow/february-2025/)

Билеты на с ледующий концерт 9 апреля 2025 года. (https://mosconcert.moscow/afisha/?month=2&year=2025)

В феврале в Москве пройдут концерты российских и зарубежных звезд.
Концерт должен пройти 28 февраля 2025 года в VK Stadium. В первые дни января 2025-го москвичи и гости столицы отправились гулять на Красную площадь. Из-за большого количества отдыхающих и силовиков, которые патрулировали центр города, попасть на главную ярмарку страны было практически невозможно. (https://msk1.ru/text/entertainment/2025/02/01/75026924/)

Посетите самые интересные и популярные мероприятия из нашей редакторской подборки «Лучшие концерты и шоу в феврале 2025». Расписание и билеты на PortalBilet.ru  8-800-500-42-62  info@portalbilet.ru... (https://portalbilet.ru/bundles/luchshie-koncerty-i-shou-v-fevrale)

Рок-концерты в Москве в феврале 2025 года. Москва. Добавить в Мои города.
Если ты не нашел выступление любимых групп в Москве в феврале 2025 года, то посмотри, какие рок-события пройдут в соседних городах. Нажимай «Добавить в Мои города», чтобы быть в курсе, когда твои любимые рок-группы будут выступать в Москве . (https://ru.myrockshows.com/location/166-moscow/feb-2025/)

Лучшие концерты Москвы ждут Вас! Билеты на концерты, подробнейшая афиша, заказ и доставка по Москве и МО. (https://biletservis.agency/concerts/february/)

Афиша концертов в Москве. Не знаете куда сходить в Москве и хотите присутствовать на всех главных музыкальных событиях вашего города? Покупайте билеты на концерты на КАССИР.РУ, не выходя из дома. Мы собираем для вас самый полный список мероприятий с достоверными ценами и актуальной информацией о наличии свободных мест.
дебюты начинающих звезд. продажа билетов на концерты популярных исполнителей. Мы представляем вашему вниманию сотни мероприятий и афишу концертов в Москве. (https://msk.kassir.ru/bilety-na-koncert/fevral)

Благотворительный фонд «Бельканто» в Москве предлагает Вам посетить концерты классической музыки в феврале 2025 года. Здесь вы найдете полную афишу классических концертов в Москве на февраль 2025 года: большой репертуар, онлайн продажа билетов, удобный поиск и подробное описание концертов. (https://www.belcantofund.com/calendar/2025/02/)

Подробное и актуальное расписание на февраль 2025 года... (https://kz-moskva.ru/afisha/february/)

Афиша концертов в Москве февраль 2025. Низкие цены на билеты от официальных продавцов. (https://rolld.ru/msk/concert/february)

Билеты на концерты в Москве 2025-2026, полное расписание выступлений артистов на всех площадках города. (https://koncert-2025.ru/moscow/month-february)

Китайский Новый год в Москве до 9 февраля. Счастливого Нового года! Празднуем по лунному календарю в самом центре столицы: на Манежной площади, Тверском бульваре и Тверской площади.
Масленица в Москве c 21 февраля. Скоро можно будет есть блины на завтрак, обед и ужин и не вызывать подозрений у окружающих. (https://gorbilet.com/blog/msk/novosti-moskvy/afisha-moskva-fevral-2025)

Концерты в Москве — это не только возможность насладиться живой музыкой, но и способ провести время в компании друзей или погрузиться в атмосферу творчества и искусства. Следите за афишей, выбирайте стили и исполнителей по вкусу, и готовьтесь к ярким впечатлениям. Собирайтесь на лучшие концерты столицы с помощью удобного онлайн-сервиса KudaGo, который поможет вам не только узнать расписание, но и приобрести билеты без лишних трудностей.
Концерты в Москве в феврале. (https://kudago.com/msk/concerts/?date=february)

«Афиша Города» Москва представляет расписание лучших концертов в твоем городе на сезон 2025-2026. События в концертных залах, стадионах, дворцах культуры, клубах и открытых сценах. Выбор места на удобной схеме зала, удобная покупка электронного билета. (https://msk.afishagoroda.ru/events/koncert)

Афиша концертов Москвы 2025–2026. Февраль.
У нас собрана самая большая афиша концертов Москвы. Мы в реальном времени найдем, где выгоднее приобрести билеты, покажем их стоимость, наличие и комиссию. Нашим пользователям не придется тратить время на то, чтобы промониторить десятки сайтов и найти самое выгодное для себя предложение. (https://musclub.ru/)

Афиша Москвы на февраль 2025 года. Все мероприятия в феврале в Москве: концерты, спектакли, спорт, для детей, фестивали, театр... (https://afisha-2025.ru/moscow/month-february)

Расписание концертов в Москве на 1 февраля 2025. Архив. Хрен, Burden Of Hatred, Потомучто, Mishouris Blues Band, Авиа, Звери, Blues Cousins, Jawa, Торба-на-Круче, Shootki, Oнвард, Atomic Cellos, HighTime Orchestra, Simple Music Ensemble, Icegergert, Владимир Кузьмин, Алёна Свиридова, Игорь Саруханов и другие музыканты. (https://rockgig.ru/schedule/2025-02-01)

Февраль в этом году радует разнообразием массовых мероприятий, на которые можно поехать с семьёй, друзьями или своей второй половинкой. По традиции многие крупные концерты и фестивали проходят в Москве и Санкт-Петербурге, но и в других регионах ожидаются интересные события.
Расскажу, какие фестивали и концерты планируются в феврале 2025-го в разных уголках России. (https://travel.yandex.ru/journal/sobytiya-etogo-fevralya/)

Февраль 2025. 1 ФЕВРАЛЯ, СУББОТА.
Авторский полнометражный художественный фильм «ЖЖ» (1час 28 минут) кинорежиссёра и писателя (роман «Штукатурное Небо») Александра И. Строева, снятый 13 лет назад на киностудии «Мосфильм» в 4 павильоне за 15 съёмочных дней на киноплёнку с оператором Максимом Шинкоренко при поддержке Министерства культуры РФ и отправленный в единственной копии на полку в хранилище Госфильмофонда в Белых столбах — зазвучал сегодня совершенно актуально и по новому —. (https://cduran.ru/%D1%84%D0%B5%D0%B2%D1%80%D0%B0%D0%BB%D1%8C-2025/)

Какие концерты будут в феврале в Москве.
1 февраля в 11:00, 14:00 Московский открытый театр кукол. (https://msk.kinoafisha.info/concerts/february/)

Московский международный Дом музыки — один из крупнейших в России филармонических комплексов, который включает в себя четыре зала: Светлановский, Камерный, Театральный и Зал Шнитке. В этих залах проходят концерты выдающихся симфонических оркестров, камерных ансамблей, артистов оперы и балета, солистов и инструменталистов. (https://www.mmdm.ru/afisha/?p_number=1&search_date_start=01-02-2025&search_date_end=28-02-2025)

Афиша концертов в Москве на февраль 2025 года, полное расписание выступлений в феврале в Москве, купите билеты онлайн на официальном сайте. (https://afisha-concert.ru/moscow/list/february)

In [17]:
query = "Кто выиграл в финале Лиги чемпионов в 2024 году?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Кто выиграл в финале Лиги чемпионов в 2024 году?

### Ответ
В финале Лиги чемпионов УЕФА 2024 года победу одержал **мадридский "Реал"**, обыгравший дортмундскую "Боруссию" со счетом 2:0.

### Источники
Финал Лиги чемпионов УЕФА 2024 года прошёл 1 июня 2024 года на стадионе «Уэмбли» в Лондоне. Этот футбольный матч стал 69-м финалом Лиги чемпионов УЕФА... (https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB_%D0%9B%D0%B8%D0%B3%D0%B8_%D1%87%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%BD%D0%BE%D0%B2_%D0%A3%D0%95%D0%A4%D0%90_2024)

Футболисты "Реала" в финале Лиги чемпионов 2023/24.
"Боруссия" третий раз в истории сыграла в решающем матче Лиги чемпионов и потерпела второе поражение. Единственный раз дортмундцы брали титул в 1997 году. После финала в Лондоне определились все победители еврокубков в сезоне 2023/24. (https://www.nur.kz/sport/football/2110740-real-pobedil-borussiyu-i-vyigral-ligu-chempionov/)

Мадридский «Реал» стал победителем Лиги чемпионов УЕФА, обыграв в финале дортмундскую «Боруссию» со счетом 2:0. Матч проходил на стадионе «Уэмбли» в Лондоне. (https://meduza.io/news/2024/06/02/real-vyigral-ligu-chempionov)

Сегодня, 1 июня, состоялся финал Лиги чемпионов 2024 года, в котором в Лондоне на стадионе «Уэмбли» встречались дортмундская «Боруссия» и мадридский «Реал». «Реал» выиграл у «Боруссии» со счетом 2:0 и стал победителем ЛЧ-2024.
Ближайший преследователь «Реала» по этому показателю — «Милан», у которого семь кубков Лиги чемпионов. (https://sport24.ru/football/news-652635-kto-vyigral-lch-pobeditel-ligi-chempionov-2024)

«Реал» выиграл Лигу чемпионов, победив в финале «Боруссию».
Таким образом, мадридский «Реал» выиграл Лигу чемпионов сезона-2023/2024. Для «сливочных» это 15-я победа в элитном европейском турнире за свою историю. Предыдущим победителем Лиги чемпионов является «Манчестер Сити», обыгравший в финале сезона-2022/2023 миланский «Интер» со счётом 1:0. Единственный гол в матче забил полузащитник «горожан» Родри. (https://www.championat.com/football/news-5574730-borussiya-real-0-2-rezultat-finala-ligi-chempionov-sezona-2034-2034-1-iyunya-2024.html)

Карло Анчелотти выиграл пятую Лигу чемпионов в качестве тренера — трижды специалист приводил к победе «Реал» (2014, 2022, 2024) и дважды итальянский «Милан» (2003, 2007). Итальянец является рекордсменом турнира среди тренеров по числу титулов, далее идут англичанин Боб Пейсли, француз Зинедин Зидан и испанец Хосеп Гвардиола, в активе которых по три титула.
Финал Лиги чемпионов в 2025 году пройдет в Мюнхене на «Альянц Арене». (https://sportmail.ru/news/football-eurocups/61334157/)

Кубок Лига чемпионов Лига Европы Лига конференций Лига наций Англия.
Победители и проигравшие общего этапа Лиги чемпионов «В этом розыгрыше "Интер" пройдет очень далеко». Антонин Кински — о последнем туре и итогах общего этапа Лиги чемпионов Жеребьевка Лиги Чемпионов и Лиги Европы. (https://www.livesport.ru/football/league/calendar/)

Мадридский "Реал" обыграл дортмундскую "Боруссию" со счетом 2:0 и стал победителем Лиги чемпионов. Финальный матч состоялся на стадионе "Уэмбли" в Лондоне. (https://ren.tv/news/sport/1226014-real-obygral-borussiiu-i-stal-pobeditelem-ligi-chempionov)

The 2024 UEFA Champions League final was the final match of the 2023–24 UEFA Champions League, the 69th season of Europe's premier club football tournament organised by UEFA, and the 32nd season since it was renamed from the European Champion Clubs' Cup to the UEFA... (https://football.fandom.com/wiki/2024_UEFA_Champions_League_Final)

Лига чемпионов УЕФА - Победитель Лиги чемпионов-2023/24: "Реал" - Новости.
Итальянец стал первым тренером, выигравшим Кубок европейских чемпионов/Лигу чемпионов четыре раза. В сезоне 2021/22 он также выиграл с "Реалом" "примеру", став первым тренером, который побеждал в чемпионатах Италии, Англии, Франции, Германии и Испании. (https://ru.uefa.com/uefachampionsleague/news/028d-1ad7ac2f2625-c3f13fc334cc-1000--%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5/)

«Реал» победил «Боруссию» и в 15-й раз выиграл Лигу чемпионов. «Реал» обыграл дортмундскую «Боруссию» в финале Лиги чемпионов.
64-летний Карло Анчелотти выиграл свою пятую Лигу чемпионов в качестве тренера — трижды с «Реалом» (2014, 2022, 2024) и дважды с итальянским «Миланом» (2003, 2007). (https://www.rbc.ru/sport/02/06/2024/665b27549a79473af88e70a7)

"Боруссия" третий раз играла в финале, дортмундцы победили в 1997 году и уступили в 2013-м.Главный тренер "Реала" Карло Анчелотти в пятый раз выиграл Лигу чемпионов, итальянец является рекордсменом по этому показателю. В 2003 и 2007 годах он побеждал с итальянским "Миланом", в 2014, 2022 и 2024-м - с "Реалом". (https://rsport.ria.ru/20240601/match-1949944744.html)

«Реал» одержал победу над «Боруссией» в финале Лиги чемпионов — 2:0.
«Боруссия» во второй раз проиграла в финале Лиги чемпионов — в сезоне-2012/13 команда под руководством Юргена Клоппа уступила «Баварии» (1:2). В единственный раз дортмундцы выиграли турнир в 1997 году. Тогда финале «Боруссия» победила «Ювентус» (3:1). (https://www.sport-express.ru/football/champions-league/news/borussiya-real-rezultat-matcha-finala-ligi-chempionov-1-iyunya-2024-2217659/)

Мадридский "Реал" стал победителем Лиги чемпионов. В финале испанский клуб со счетом 2:0 обыграл дортмундскую "Боруссию". (https://rg.ru/2024/06/02/madridskij-real-v-15-j-raz-vyigral-ligu-chempionov.html)

Для тренера мадридцев Карло Анчелотти это пятая победа в Лиге чемпионов. Итальянец трижды брал трофей с «Реалом» (2014, 2022 и 2024), еще два — с «Миланом» (2003, 2007).
Клуб выигрывал Лигу чемпионов в 1997 году, в финале того турнира команда одолела итальянский «Ювентус» (3:1). В 2013 году в финале ЛЧ дортмундцы уступили «Баварии» (1:2). (https://www.forbes.ru/sport/513835-real-vyigral-ligu-cempionov-v-15-j-raz)

Завершился финал Лиги чемпионов УЕФА сезона 2023-2024, в котором встречались дортмундская "Боруссия" и мадридский "Реал". Команды сыграли на стадионе "Уэмбли" в Лондоне. Встреча завершилась победой "королевского клуба". На 74-й минуте счет в игре открыл защитник "Реала" Даниэль Карвахаль. (https://smotrim.ru/article/3979922)

1 июня состоялся финал футбольной Лиги чемпионов. Мадридский «Реал» обыграл дортмундскую «Боруссию» и в 15-й раз в своей истории стал победителем турнира.
Еще до старта нынешнего розыгрыша Лиги чемпионов мадридский «Реал» по числу титулов в этом турнире превосходил следующий за ним «Милан» в два раза. (https://www.olympics.com/ru/news/futbol-liga-chempionov-2023-2024-madridskii-real-vyigral-turnir-v-15-i-raz)

В Лондоне завершился финальный матч Лиги чемпионов , в котором мадридский «Реал» в очередной раз стал сильнейшей командой Европы. (https://www.sports.ru/football/1116112323-obzor-matcha-real-madrid-borussiya-dortmund-20-final-ligi-chempionov-2.html)

Что случилось в финале Лиги чемпионов «Боруссия» — «Реал».
«Реал» победил дортмундскую «Боруссию» со счетом 2:0 в финале Лиги чемпионов в Лондоне на «Уэмбли». первый гол с передачи Тони Крооса забил Дани Карвахаль, а второй — Винисиус. Для мадридского клуба это 15-я победа в престижном еврокубке. (https://www.kp.ru/sports/futbol/chto-sluchilos-v-finale-ligi-chempionov-borussiya-real-2024/)

Финал Лиги чемпионов УЕФА 2024 года прошёл 1 июня 2024 года на стадионе «Уэмбли» в Лондоне. За титул боролись немецкая «Боруссия Дортмунд» и испанский «Реал Мадрид». Одержав победу со счётом 2:0... (https://ru.ruwiki.ru/wiki/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB_%D0%9B%D0%B8%D0%B3%D0%B8_%D1%87%D0%B5%D0%BC%D0%BF%D0%B8%D0%BE%D0%BD%D0%BE%D0%B2_%D0%A3%D0%95%D0%A4%D0%90_2024)

Рассказываем, кто выиграл Лигу Чемпионов в 2024 году и как проходил финал главного футбольного турнира. В очередной раз продемонстрировав свой высочайший уровень, «Реал Мадрид» триумфально завершил кампанию Лиги Чемпионов, одержав победу над «Боруссией Дортмунд» в финальном матче со счетом 2:0. Финал, который состоялся на легендарном стадионе "Уэмбли" в Лондоне, собрал миллионы болельщиков по всему миру у экранов телевизоров и тысячами на трибунах, стал настоящим праздником футбола. (https://spb.tsargrad.tv/news/kto-vyigral-ligu-chempionov-2024-kak-prohodil-final-glavnogo-futbolnogo-turnira_1008781)

Мадридский «Реал» обыграл дортмундскую «Боруссию» в финале Лиги чемпионов.
При этом за 3 минуты до конца основного времени рефери не засчитал гол немецкой команды. Предыдущим победителем Лиги чемпионов был «Манчестер Сити», обыгравший в финале миланский «Интер» со счётом 1:0. «Реал» выиграл Лигу чемпионов в 15-й раз. (https://www.mk.ru/sport/2024/06/01/madridskiy-real-stal-pobeditelem-ligi-chempionov.html)

Мадридский «Реал» одержал победу над дортмундской «Боруссии» в финале Лиги чемпионов 2024 со счетом 2:0. Испанский клуб стал лучшим в Европе в 15-й раз. (https://vtomske.ru/news/204646-real-vyigral-final-ligi-chempionov-2024)

Финальный матч Лиги Чемпионов 2024 года, состоявшийся 1 июня, стал настоящей битвой титанов.
Хотя награду лучшему игроку Лиги Чемпионов 2024 года получил Криштиану Роналду на церемонии жеребьевки Лиги чемпионов УЕФА-2024/25, стоит отметить вклад каждого игрока «Реала» в эту победу. (https://telegra.ph/Kto-pobedil-LCH-2024-Triumf-Real-Madrid-v-Lige-CHempionov-2024-Istoriya-Geroi-i-Detali-Pobedy-12-28)

Лига чемпионов УЕФА 2024/2025: таблица и календарь, список бомбардиров и статистика игроков, обзоры матчей и видео голов, прогнозы и ставки на матчи. (https://soccer365.ru/competitions/19/)

«Реал» выиграл Лигу чемпионов. 19 015. 02 июня 2024 в 9:25.
Мадридский «Реал» обыграл дортмундскую «Боруссию» со счетом 2:0 в финале Лиги чемпионов на лондонском «Уэмбли». На 74-й минуте счет открыл защитник «Реала» Даниэль Карвахаль. На 83-й минуте нападающий Винисиус Жуниор забил еще один гол. (https://people.onliner.by/2024/06/02/real-vyigral-ligu-chempionov)

«Реал» обыграл «Боруссию» и стал победителем Лиги чемпионов.
Уточняется, что «Реал» в 15-й раз стал победителем турнира и упрочил лидерство по этому показателю, всего он играл в финалах Лиги чемпионов рекордные 18 раз. Второе место занимает итальянский «Милан» (семь побед), на третьей строчке — английский «Ливерпуль» и германская «Бавария» (по шесть побед). (https://news.ru/football/opredelilsya-pobeditel-ligi-chempionov/)

«Реал» победил «Боруссию» и выиграл Лигу чемпионов сезона-2023/2024.
Мадридский «Реал» победил дортмундскую «Боруссию» в финале Лиги чемпионов сезона-2023/2024. Об этом сообщает корреспондент «Ленты.ру». Игра, фаворитами которой считались мадридцы, прошла на стадионе «Уэмбли» в Лондоне (Англия) и завершилась со счетом 2:0. Забитыми мячами отметились Даниэль Карвахаль (74-я минута) и Винисиус Жуниор (83-я минута). (https://lenta.ru/news/2024/06/01/real-v-15-y-raz-vyigral-ligu-chempionov/)

"Боруссия" третий раз играла в финале, дортмундцы победили в 1997 году и уступили в 2013-м.
Защитник мадридского "Реала" Дани Карвахаль признан самым ценным игроком финала футбольной Лиги чемпионов. "Реал" является рекордсменом по количеству побед в Лиге чемпионов - мадридский клуб выиграл трофей в 15-й раз (1956-1960, 1966, 1998, 2000, 2002, 2014, 2016-2018, 2022, 2024), команда 18-й раз сыграла в матче за трофей. (https://ru.sputnik.kz/20240602/real-obygral-borussiyu-i-stal-pobeditelem-ligi-chempionov-44712099.html)

Что касается финала Лиги Чемпионов, то в последний раз немецкая команда добиралась до него в сезоне 2012/13, где уступила другому клубу из Германии - мюнхенской «Баварии».
До этого он уже дважды побеждал в турнире с «Реалом» в 2014 и 2022 гг., а также с «Миланом» в 2003 и 2007 гг. Что касается ключевых игроков королевского клуба - Луки Модрича, Начо, Тони Крооса и Дани Карвахаля, то для первых двух Кубок Лиги Чемпионов стал двадцать шестым трофеем в составе «Реала», а для вторых - шестым. (https://dzen.ru/a/ZludesMrE1jQHafT)

In [19]:
query = "Что такое LangChain?"
response = chain_with_source.invoke({"question": query})
final_answer = generate_final_answer(response)
Markdown(final_answer)

### Вопрос
Что такое LangChain?

### Ответ
LangChain — это **фреймворк с открытым исходным кодом** для разработки приложений на основе больших языковых моделей (LLM). Он предоставляет разработчикам инструменты для интеграции LLMs с внешними данными, такими как базы данных, файлы PDF, электронные письма и другие источники. С помощью LangChain можно создавать различные приложения, включая чат-боты, системы генерации текста, агенты для выполнения сложных задач и многое другое. 

Основные особенности LangChain:
1. **Композиция**: Позволяет объединять несколько LLMs и внешних инструментов в цепочки (chains) для решения комплексных задач.
2. **Работа с промптами**: Предоставляет функции для удобного представления, форматирования и управления промптами.
3. **Интеграции**: Поддерживает интеграцию с различными источниками данных и сервисами, облегчая доступ моделей к необходимой информации.
4. **Упрощение экспериментов**: Делает проще процесс тестирования разных LLMs, позволяя заменять одну модель на другую с минимальными изменениями в коде.
5. **Поддержка памяти**: Включает компоненты для управления предыдущими сообщениями в чате и включения их в цепочки действий.
   
LangChain активно развивается сообществом, имеет обширную документацию и множество примеров, что делает его мощным инструментом для разработчиков в области искусственного интеллекта.

### Источники
LangChain is a composable framework to build with LLMs.
“Working with LangChain and LangSmith on the Elastic AI Assistant had a significant positive impact on the overall pace and quality of the development and shipping experience. (https://www.langchain.com/)

Промпты: LangChain предоставляет ряд функции для работы с промптами – представление промпта согласно типу модели, формировани шаблона на основе внешних данных, форматирование вывода модели. Индексы: Индексы структурируют документы для оптимального взаимодействия с языковыми моделями. (https://habr.com/ru/articles/729664/)

LangChain is an open source framework in AI development that helps developers to build and deploy AI models and LLM easily... (https://medium.com/ai-for-absolute-beginners/what-is-langchain-explained-in-everyday-langauge-for-ai-beginners-fc8ffa6fb5f2)

Langchain позволяет интегрировать большие языковые модели с собственными данными, например, с базой данных Notion, PDF-файлами или электронными письмами. Фреймворк упрощает создание приложений, предоставляя возможности компоновки и шесть ключевых функциональных модулей. (https://tproger.ru/articles/chto-takoe-langchain---frejmvork-python-dlya-ii)

LangChain был запущен в октябре 2022 года как проект с открытым исходным кодом Харрисоном Чейзом, работавшим в стартапе машинного обучения Robust Intelligence. Проект быстро завоевал популярность[3] благодаря улучшениям от сотен участников на GitHub, актуальным обсуждениям в Twitter, оживленной активности на сервере Discord проекта, множеству руководств YouTube и встречам в Сан-Франциско и Лондоне. (https://tr-page.yandex.ru/translate?lang=en-ru&url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FLangChain)

LangChain is a framework for developing applications powered by large language models (LLMs). (https://langchainjs-docs-ruddy.vercel.app/docs/introduction/)

LangChain – это платформа с открытым исходным кодом для создания приложений на основе больших языковых моделей (LLM).
С помощью LangChain организации могут без переобучения или доработки перепрофилировать большие языковые модели для приложений, нацеленных на узкую предметную область. (https://aws.amazon.com/ru/what-is/langchain/)

Important integrations have been split into lightweight packages that are co-maintained by the LangChain team and the integration developers. langchain: Chains, agents, and retrieval strategies that make up an application's cognitive architecture. (https://github.com/langchain-ai/langchain/blob/master/README.md)

Используйте модели, обслуживаемые Databricks, в качестве LLM или внедрения в приложение LangChain. Интеграция векторного поиска вектора Мозаики для хранения векторов и извлечения. Управление моделями LangChain и производительность в экспериментах MLflow. (https://learn.microsoft.com/ru-ru/azure/databricks/large-language-models/langchain)

While these integrations can generally be achieved with fully manual code, orchestration frameworks such as LangChain and the IBM watsonx portfolio of AI products greatly simplify the process. They also make it much easier to experiment with different LLMs to compare results, as different models can be swapped in and out with minimal changes to code. (https://www.ibm.com/think/topics/langchain)

LangChain provides several classes and functions to make constructing and working with prompts easy.
LangChain provides memory components to manage and manipulate previous chat messages and incorporate them into chains. (https://blog.apify.com/what-is-langchain/)

LangChain — это фреймворк, находящийся в свободном доступе.
LangChain революционизирует процесс разработки широкого спектра приложений, включая чат-боты, генеративные вопросно-ответные системы и генераторы информационных обзоров текстовых и аудиовизуальных материалов. (https://dzen.ru/a/ZVYtQkMxmhivgLek)

LangChain aids in implementing the retrieval-augmented generation (RAG) pattern in applications, simplifying data retrieval from various sources.
LangChain is a Python framework designed to streamline AI application development, focusing on real-time data processing and integration with large language models (LLMs). (https://www.datastax.com/guides/what-is-langchain)

In simple terms, LangChain is a standardized interface that simplifies the process of building AI apps.
When a user interacts with the app, LangChain uses its connections to an LLM to process the request and generate appropriate responses. (https://www.elastic.co/blog/langchain-tutorial)

LangChain is a model-agnostic, open-source framework that gives developers the flexibility to experiment with various LLMs to discover the optimal fit for their unique applications. It offers a generic interface to a range of foundational models. (https://builtin.com/articles/langchain)

Explore the comprehensive guide for LangChain, covering all technical aspects and functionalities. (https://www.restack.io/docs/langchain-knowledge-langchain-documentation-guide)

LangChain is a programming language platform that lets developers construct and connect models to access, transform, and share data seamlessly. It offers a powerful and versatile environment for model development, enabling the manipulation of data structures, model chaining, and the incorporation of external agents like LaMDA. (https://cloud.google.com/use-cases/langchain)

LangChain appeared around the same time. Its creator, Harrison Chase, made the first commit in late October 2022.
The Hugging Face Hub endpoint in LangChain connects to the Hugging Face Hub and runs the models via their free inference endpoints. (https://www.pinecone.io/learn/series/langchain/langchain-intro/)

The Strength of LangChain Chains. Chains Without LLMs and Prompts. Crafting Chains: The SimpleSequentialChain.
LangChain is a captivating tool that is making a significant impact in the dynamic sphere of language learning models (LLMs). (https://ecoagi.ai/articles/langchain-chains-what-is-langchain)

LangChain — это универсальная библиотека Python , которая позволяет разработчикам и исследователям создавать, экспериментировать и анализировать языковые модели и агенты.
В LangChain агент — это сущность, которая может понимать и генерировать текст. (https://bestcodes.ru/backend/polnoe-rukovodstvo-po-langchain-v-python/)

LangChain is a software development framework that makes it easier to create applications using large language models (LLMs).
LangChain allows developers to combine LLMs like GPT-4 with external data, opening up possibilities for various applications such as chatbots, code understanding, summarization, and more. (https://analyzingalpha.com/langchain-python-tutorial)

LangChain — это фреймворк для разработки приложений на основе языковых моделей. Мы считаем, что самые мощные и дифференцированные приложения будут не только обращаться к языковой модели через API, но также будут: Будьте осведомлены о данных : подключите языковую модель к другим источникам данных Будьте агентом : позвольте языковой модели взаимодействовать с окружающей средой. (https://www.youtube.com/watch?v=2u_O3uovpzY)

This means LangChain applications can understand the context, such as prompt instructions or content grounding responses and use Large Language Models for complex reasoning tasks, like deciding how to respond or what actions to take. (https://nanonets.com/blog/langchain/)

Let's dive into LangChain in detail to show you how it works, what developers can build with it, and how it fits into ML architectures. (https://lakefs.io/blog/what-is-langchain-ml-architecture/)

LangChain offers features designed to streamline the development of AI applications.
Alongside LangChain's open-source libraries and third-party integrations, you'll be well-equipped to enhance your business operations with high-quality, AI-powered solutions. (https://www.voiceflow.com/articles/langchain)

LangChain — это мощный фреймворк, созданный для обработки естественного языка (Natural Language Processing, NLP) и глубокого обучения (Deep Learning) на языке программирования Python.
Фреймворк LangChain дает разработчику богатый функционал для работы с различными файлами данных и задачами машинного обучения. (https://ya.zerocoder.ru/chto-takoe-langchain-frejmvork-python-dlya-ii/)

Chains: Chains go beyond just a single LLM call, and are sequences of calls (whether to an LLM or a different utility). LangChain provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications. (https://langchain-doc.readthedocs.io/en/latest/index.html)

LangChain - это мощная структура, которая интегрируется с внешними инструментами для формирования экосистемы.
LangChain позволяет легко хранить и извлекать векторы из различных источников, начиная от массивов в памяти и заканчивая размещенными векторными базами данных, такими как Pinecone. (https://telegra.ph/Kratkoe-rukovodstvo-po-LangChain-dlya-razrabotchikov-programmnogo-obespecheniya-08-30)

Как упоминалось выше, после того как LangChain получает текстовое содержимое, возвращенное большой моделью (LLM), он может использовать специальные парсеры вывода для форматирования текстового содержимого, такие как разбор JSON или преобразование вывода LLM в объект Python. (https://www.alldevstack.com/ru/python-langchain-tutorial/langchain-introduction.html)

LangChain позволяет моделям LLM генерировать ответы на основе самой актуальной информации, доступной в сети, а также упрощает процесс организации больших объемов данных для легкого доступа моделями LLM. (https://professiy.com/guides/chto-takoe-langchain-i-kak-on-rabotaet-rukovodstvo-dlya-nachinayushhih)